# 基于财务数据的机器学习选股与回测（HS300）—季度预测（终版）

这个 Notebook 实现：

- 以财务报表“公告日”为信息可得时间，对齐到**季度末调仓日**（只用当时已公告的财务信息）
- 预测未来约 **60 个交易日（≈1个季度）**收益
- 采用滚动（扩展窗口）训练 + 每季度末调仓 TopK 等权多头
- **训练标签可做截尾（clip）稳定训练，但回测收益一律使用真实收益 `y_raw`**

> 你需要在本机能使用 `baostock` 下载行情；如果你已经有价格缓存，会自动复用。

In [ ]:

# 0. 环境与依赖
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.ensemble import HistGradientBoostingRegressor

pd.set_option("display.width", 140)
pd.set_option("display.max_columns", 60)

## 1. 配置参数

In [ ]:

# 1) 财务数据路径（你自己的 pkl / csv 都行；默认给出示例变量名）
FIN_PATH = "hs300_financial_quarterly_all.pkl"   # <- 修改成你的实际路径

# 2) 价格缓存（parquet）
PRICE_CACHE = "hs300_daily_prices.parquet"

# 3) 复权方式：建议先用不复权(1)调试，再用后复权(3)做最终
ADJUSTFLAG = "3"     # "1"不复权, "2"前复权, "3"后复权

# 4) 预测周期（交易日）
HORIZON = 60

# 5) 策略参数
TOPK = 30
MIN_TRAIN_QUARTERS = 12   # 至少多少个季度点才开始训练/回测（42个季度样本，12较稳）

# 6) 训练标签截尾分位（只用于训练，不用于回测收益）
Y_CLIP_LO = 0.01
Y_CLIP_HI = 0.99

# 7) （可选）过滤极端真实收益（数据异常时建议开启）
FILTER_Y_RAW = True
Y_RAW_LO, Y_RAW_HI = -0.8, 5.0   # 真实收益过滤阈值，可按需要调整

## 2. 读取财务数据并清洗

假设财务数据至少包含：
- `code`：股票代码（baostock 格式例如 `sh.600000`）
- `announce_date`：公告日（或可由 `pubDate`/`pubdate` 转换得到）
- `statDate`：报表期末日（例如 2014-09-30）
- 大量数值型财务指标列（作为特征）

如果你的列名不同，在下面“列名适配”位置改一下即可。

In [ ]:

# 读取财务数据
if FIN_PATH.endswith(".pkl"):
    fin = pd.read_pickle(FIN_PATH)
else:
    fin = pd.read_csv(FIN_PATH)

print(fin.shape)
fin.head()

In [ ]:

# 列名适配：把你数据里的公告日/报表期字段对齐到 announce_date / statDate
fin = fin.copy()

# 尝试自动识别公告日字段
if "announce_date" not in fin.columns:
    for cand in ["pubDate", "pubdate", "ann_date", "announceDate", "announcement_date"]:
        if cand in fin.columns:
            fin = fin.rename(columns={cand: "announce_date"})
            break

# 尝试自动识别报表期字段
if "statDate" not in fin.columns:
    for cand in ["stat_date", "report_period", "reportDate", "endDate"]:
        if cand in fin.columns:
            fin = fin.rename(columns={cand: "statDate"})
            break

need_cols = ["code", "announce_date", "statDate"]
missing = [c for c in need_cols if c not in fin.columns]
if missing:
    raise KeyError(f"财务数据缺少必要字段：{missing}。请在上面列名适配处修改。")

fin["announce_date"] = pd.to_datetime(fin["announce_date"])
fin["statDate"] = pd.to_datetime(fin["statDate"])

if "year" not in fin.columns:
    fin["year"] = fin["statDate"].dt.year
if "quarter" not in fin.columns:
    fin["quarter"] = fin["statDate"].dt.quarter

fin = fin.sort_values(["code","announce_date"]).drop_duplicates(["code","announce_date"], keep="last").reset_index(drop=True)

print("after clean:", fin.shape)
fin[["code","announce_date","statDate","year","quarter"]].head()

In [ ]:

# 选择特征列：数值型列（排除标识符/日期列）
id_cols = {"code","announce_date","statDate","year","quarter"}
numeric_cols = [c for c in fin.columns if c not in id_cols and pd.api.types.is_numeric_dtype(fin[c])]
feature_cols = numeric_cols
print("num features:", len(feature_cols))
feature_cols[:20]

## 3. 下载/读取日线价格（用于计算未来 60 交易日收益）

- 若存在 `PRICE_CACHE`，将直接读取
- 否则用 baostock 下载日线收盘价并缓存

> 注意：这里下载的是 **全样本区间**，可能比较耗时；首次跑需要耐心。

In [ ]:

def download_hs300_prices_baostock(codes, start_date, end_date, cache_path=PRICE_CACHE, adjustflag="3"):
    import baostock as bs

    lg = bs.login()
    if lg.error_code != "0":
        raise RuntimeError("baostock login failed")

    rows = []
    for code in tqdm(codes, desc="Download prices"):
        rs = bs.query_history_k_data_plus(
            code,
            "date,code,close",
            start_date=start_date,
            end_date=end_date,
            frequency="d",
            adjustflag=adjustflag
        )
        if rs.error_code != "0":
            continue
        data_list = []
        while (rs.error_code == "0") and rs.next():
            data_list.append(rs.get_row_data())
        if not data_list:
            continue
        df = pd.DataFrame(data_list, columns=rs.fields)
        rows.append(df)

    bs.logout()

    if not rows:
        raise RuntimeError("no price data downloaded")

    prices = pd.concat(rows, ignore_index=True)
    prices["date"] = pd.to_datetime(prices["date"])
    prices["close"] = pd.to_numeric(prices["close"], errors="coerce")
    prices = prices.dropna(subset=["date","close"])
    prices = prices.sort_values(["code","date"]).reset_index(drop=True)

    prices.to_parquet(cache_path, index=False)
    return prices

codes = sorted(fin["code"].unique())
start_date = fin["announce_date"].min().strftime("%Y-%m-%d")
end_date   = (fin["announce_date"].max() + pd.Timedelta(days=400)).strftime("%Y-%m-%d")
print("codes:", len(codes), "range:", start_date, "->", end_date)

if os.path.exists(PRICE_CACHE):
    prices = pd.read_parquet(PRICE_CACHE)
    prices["date"] = pd.to_datetime(prices["date"])
    prices["close"] = pd.to_numeric(prices["close"], errors="coerce")
    prices = prices.dropna(subset=["date","close"]).sort_values(["code","date"]).reset_index(drop=True)
    print("Loaded cache:", prices.shape, "from", PRICE_CACHE)
else:
    prices = download_hs300_prices_baostock(codes, start_date, end_date, cache_path=PRICE_CACHE, adjustflag=ADJUSTFLAG)
    print("Downloaded:", prices.shape)

prices.head()

## 4. 构造季度末调仓面板：对齐公告日（避免未来函数）

- 生成季度末日期序列（调仓日）
- 对每只股票做 `merge_asof`：在季度末 `date` 向后匹配最近一次 `announce_date` 的财务数据（backward）

In [ ]:

def build_quarterly_panel(fin_df: pd.DataFrame, feat_cols):
    fin_df = fin_df.sort_values(["code","announce_date"]).copy()

    q_ends = pd.date_range(
        fin_df["announce_date"].min().to_period("Q").to_timestamp("Q"),
        fin_df["announce_date"].max().to_period("Q").to_timestamp("Q"),
        freq="Q"
    )

    panels = []
    for code, g in tqdm(fin_df.groupby("code"), desc="Build quarterly panel"):
        tmp = pd.DataFrame({"date": q_ends})
        tmp["code"] = code
        g2 = g.drop(columns=["code"], errors="ignore").sort_values("announce_date")

        mg = pd.merge_asof(
            tmp.sort_values("date"),
            g2,
            left_on="date",
            right_on="announce_date",
            direction="backward"
        )
        panels.append(mg)

    panel = pd.concat(panels, ignore_index=True)
    panel = panel.dropna(subset=["announce_date"]).copy()

    keep_cols = ["date","code","announce_date","statDate","year","quarter"] + feat_cols
    panel = panel[keep_cols].sort_values(["date","code"]).reset_index(drop=True)
    return panel

panel = build_quarterly_panel(fin, feature_cols)
print(panel.shape, panel["date"].min(), panel["date"].max())
panel.head()

## 5. 用调仓日计算未来 60 交易日真实收益 `y_raw`（稳健版）

- 入场价：调仓日之后的**第一个交易日**收盘价（`direction="forward"`）
- 出场价：该交易日往后 **H=60** 个交易日的收盘价（通过 `shift(-H)`）
- 真实收益：`y_raw = close_fwd / close - 1`

⚠️ `merge_asof` 要求 **on 键（date）在 DataFrame 全局有序**，所以必须 `sort_values(["date","code"])`。

In [ ]:

def attach_forward_return(panel_df: pd.DataFrame, prices_df: pd.DataFrame, horizon=60):
    prices2 = prices_df.copy()
    prices2["date"] = pd.to_datetime(prices2["date"])
    prices2["close"] = pd.to_numeric(prices2["close"], errors="coerce")
    prices2 = prices2.dropna(subset=["date","close"]).sort_values(["code","date"]).reset_index(drop=True)

    prices2["close_fwd"] = prices2.groupby("code")["close"].shift(-horizon)

    events = panel_df[["code","date"]].copy()
    events["date"] = pd.to_datetime(events["date"])

    events = events.sort_values(["date","code"]).reset_index(drop=True)
    right = prices2[["code","date","close","close_fwd"]].sort_values(["date","code"]).reset_index(drop=True)

    aligned = pd.merge_asof(
        events,
        right,
        on="date",
        by="code",
        direction="forward",
        allow_exact_matches=True
    )

    aligned["y_raw"] = aligned["close_fwd"] / aligned["close"] - 1
    out = panel_df.merge(aligned[["code","date","y_raw"]], on=["code","date"], how="left")
    return out

panel = attach_forward_return(panel, prices, horizon=HORIZON)
print("y_raw negatives:", (panel["y_raw"]<0).sum(), "min:", panel["y_raw"].min(), "max:", panel["y_raw"].max())
panel[["date","code","y_raw"]].head()

In [ ]:

# 可选：过滤明显异常的真实收益（避免少数离群点污染训练/回测）
if FILTER_Y_RAW:
    before = len(panel)
    panel = panel[(panel["y_raw"] > Y_RAW_LO) & (panel["y_raw"] < Y_RAW_HI)].copy()
    after = len(panel)
    print(f"FILTER_Y_RAW: {before} -> {after} (dropped {before-after})")

# 训练标签 y 默认等于真实收益（后面训练时会对训练集 clip）
panel["y"] = panel["y_raw"]

panel[["y_raw","y"]].describe([.01,.05,.5,.95,.99])

## 6. 特征处理：按日期截面去极值 + 缺失值填充

- 对特征做按 date 截面 winsorize（1%~99%）
- 缺失值：先按 date 填中位数，再用全局中位数兜底

In [ ]:

def winsorize_by_date(df: pd.DataFrame, cols, lower=0.01, upper=0.99):
    df = df.copy()
    for d, idx in tqdm(df.groupby("date").groups.items(), desc="Winsorize features"):
        x = df.loc[idx, cols]
        lo = x.quantile(lower)
        hi = x.quantile(upper)
        df.loc[idx, cols] = x.clip(lo, hi, axis=1)
    return df

panel2 = winsorize_by_date(panel, feature_cols, 0.01, 0.99)

panel2[feature_cols] = panel2.groupby("date")[feature_cols].transform(lambda x: x.fillna(x.median()))
panel2[feature_cols] = panel2[feature_cols].fillna(panel2[feature_cols].median())

panel2 = panel2.dropna(subset=["y_raw"]).copy()

print(panel2.shape)
panel2.head()

## 7. 滚动回测（季度末调仓）

- 训练时：对 `y_raw` 做 clip（只影响训练稳定性）
- 回测收益：用 `y_raw`（真实收益）计算组合/基准收益
- IC：用 `y_raw` 衡量预测分数与真实收益的 Spearman 相关

In [ ]:

def perf_stats(returns: pd.Series, freq=4):
    r = returns.dropna()
    if len(r) == 0:
        return {}
    cum = (1 + r).cumprod()
    peak = cum.cummax()
    dd = cum / peak - 1
    ann_ret = (cum.iloc[-1]) ** (freq / len(r)) - 1
    ann_vol = r.std(ddof=1) * np.sqrt(freq)
    sharpe = (r.mean() / r.std(ddof=1)) * np.sqrt(freq) if r.std(ddof=1) > 0 else float("nan")
    return {"ann_ret": ann_ret, "ann_vol": ann_vol, "sharpe": sharpe, "max_drawdown": dd.min()}

def walk_forward_backtest(panel_df: pd.DataFrame,
                          feat_cols,
                          topk=30,
                          min_train_quarters=12,
                          y_clip_lo=0.01,
                          y_clip_hi=0.99):
    panel_df = panel_df.sort_values(["date","code"]).copy()
    dates = sorted(panel_df["date"].unique())  # 已经是季度末

    preds_list = []
    port_rets = []
    bench_rets = []
    ic_list = []

    for d in tqdm(dates, desc="Walk-forward (quarterly)"):
        train = panel_df[panel_df["date"] < d]
        test  = panel_df[panel_df["date"] == d]

        if train["date"].nunique() < min_train_quarters:
            continue
        if test.empty:
            continue

        X_train = train[feat_cols].values
        y_train_raw = train["y_raw"].values

        # 训练标签截尾（只用于训练）
        lo = np.nanquantile(y_train_raw, y_clip_lo)
        hi = np.nanquantile(y_train_raw, y_clip_hi)
        y_train = np.clip(y_train_raw, lo, hi)

        X_test = test[feat_cols].values

        model = HistGradientBoostingRegressor(max_depth=6, learning_rate=0.05, max_iter=300)
        model.fit(X_train, y_train)
        score = model.predict(X_test)

        tmp = test[["date","code","y_raw"]].copy()
        tmp["score"] = score
        preds_list.append(tmp)

        ic = tmp[["score","y_raw"]].corr(method="spearman").iloc[0,1]
        ic_list.append({"date": d, "ic": ic})

        tmp2 = tmp.sort_values("score", ascending=False)
        long = tmp2.head(topk)
        port_ret = long["y_raw"].mean()
        bench_ret = tmp["y_raw"].mean()

        port_rets.append({"date": d, "ret": port_ret})
        bench_rets.append({"date": d, "ret": bench_ret})

    preds = pd.concat(preds_list, ignore_index=True) if preds_list else pd.DataFrame()
    port = pd.DataFrame(port_rets).set_index("date")["ret"] if port_rets else pd.Series(dtype=float)
    bench = pd.DataFrame(bench_rets).set_index("date")["ret"] if bench_rets else pd.Series(dtype=float)
    ic_s = pd.DataFrame(ic_list).set_index("date")["ic"] if ic_list else pd.Series(dtype=float)

    return preds, port, bench, ic_s

preds, port, bench, ic = walk_forward_backtest(
    panel2, feature_cols,
    topk=TOPK,
    min_train_quarters=MIN_TRAIN_QUARTERS,
    y_clip_lo=Y_CLIP_LO,
    y_clip_hi=Y_CLIP_HI
)

print("quarters:", len(port), port.index.min(), port.index.max())
print("port negatives:", (port<0).sum(), "min:", port.min())
print("bench negatives:", (bench<0).sum(), "min:", bench.min())

In [ ]:

summary = pd.DataFrame([
    {"strategy":"ML_TopK", **perf_stats(port, freq=4), "mean_ic": ic.mean()},
    {"strategy":"Benchmark_EW", **perf_stats(bench, freq=4), "mean_ic": np.nan},
])
summary

In [ ]:

cum_port = (1 + port).cumprod()
cum_bench = (1 + bench).cumprod()

plt.figure(figsize=(9,4))
plt.plot(cum_port.index, cum_port.values, label="ML TopK (Q)")
plt.plot(cum_bench.index, cum_bench.values, label="Equal-weight Benchmark (Q)")
plt.legend()
plt.title("Cumulative Return (Quarterly rebalance)")
plt.tight_layout()
plt.show()

plt.figure(figsize=(9,3))
plt.plot(ic.index, ic.values)
plt.axhline(0, linestyle="--")
plt.title(f"IC (Spearman), mean={ic.mean():.4f}")
plt.tight_layout()
plt.show()

## 8. 可选输出：每期持仓、个股贡献

- `preds`：每期每只股票的 score 与真实收益 `y_raw`
- 根据 score 排序取 TopK 得到持仓表

In [ ]:

holdings = (
    preds.sort_values(["date","score"], ascending=[True, False])
         .groupby("date")
         .head(TOPK)
         .assign(weight=1.0/TOPK)
)
holdings.head(20)

In [ ]:

holdings["contrib"] = holdings["weight"] * holdings["y_raw"]
stock_contrib = holdings.groupby("code")["contrib"].sum().sort_values(ascending=False)
stock_contrib.head(30)

In [ ]:

out_dir = "outputs"
os.makedirs(out_dir, exist_ok=True)

summary.to_csv(os.path.join(out_dir, "summary.csv"), index=False)
preds.to_csv(os.path.join(out_dir, "preds_by_quarter.csv"), index=False)
holdings.to_csv(os.path.join(out_dir, "holdings_topk.csv"), index=False)

print("saved to", out_dir)